In [1]:
from intuitlib.client import AuthClient

auth_client = AuthClient(
        client_id='ABVlDV6VbSq3SyfN6ShES7gauQ8LExHxnhuoDVsKaEOe3GmBrT',
        client_secret='zqQX950hO9CICCreDvfAvk3mE7tuJJKrGyyrRxEU',
        access_token=None,  # If you do not pass this in, the Quickbooks client will call refresh and get a new access token. 
        environment='production',
        redirect_uri='https://developer.intuit.com/v2/OAuth2Playground/RedirectUrl',
    )

In [2]:
auth_client

In [3]:
from quickbooks import QuickBooks

client = QuickBooks(
        auth_client=auth_client,
        refresh_token='AB11689288216Bnpsdbbh5oJKWBLVwTTHCN5OAhL7FmkVkzkUJ',
        company_id='514655250',
    )

In [4]:
from quickbooks.objects.customer import Customer
customers = Customer.all(qb=client)

In [6]:
customers[0].__dict__

{'Id': '222',
 'SyncToken': '0',
 'sparse': False,
 'domain': 'QBO',
 'Title': '',
 'GivenName': 'Adam',
 'MiddleName': '',
 'FamilyName': 'Fox',
 'Suffix': '',
 'FullyQualifiedName': 'Adam Fox',
 'CompanyName': '',
 'DisplayName': 'Adam Fox',
 'PrintOnCheckName': 'Adam Fox',
 'Notes': '',
 'Active': True,
 'IsProject': False,
 'Job': False,
 'BillWithParent': False,
 'Taxable': False,
 'Balance': 0,
 'BalanceWithJobs': 0,
 'PreferredDeliveryMethod': 'Print',
 'ResaleNum': '',
 'Level': 0,
 'OpenBalanceDate': '',
 'PrimaryTaxIdentifier': '',
 'BillAddr': <quickbooks.objects.base.Address at 0x264e9cd4670>,
 'ShipAddr': <quickbooks.objects.base.Address at 0x264e9cd4f40>,
 'PrimaryPhone': None,
 'AlternatePhone': None,
 'Mobile': None,
 'Fax': None,
 'PrimaryEmailAddr': None,
 'WebAddr': None,
 'DefaultTaxCodeRef': None,
 'SalesTermRef': None,
 'PaymentMethodRef': None,
 'ParentRef': None,
 'ARAccountRef': None,
 'CurrencyRef': <quickbooks.objects.base.Ref at 0x264e9d2cc70>,
 'MetaData': 